In [ ]:
!pip install Together
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 462.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 11.9 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.3.0 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import torch
import time
import datasets
from together import Together
import json

In [ ]:
def set_seed(seed: int) -> None:
  """
  Set the random seed for reproducibility.

  Args:
    seed (int): The seed value to set.

  Returns:
    None
  """

  import os
  import random
  import numpy as np

  np.random.seed(seed)
  random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  # When running on the CuDNN backend, two further options must be set
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  # Set a fixed value for the hash seed
  os.environ["PYTHONHASHSEED"] = str(seed)
  print(f"Random seed set as {seed}")

set_seed(42)

Random seed set as 42


In [ ]:
train_data = datasets.load_dataset("art", split="train").shuffle(seed=42)
test_data = datasets.load_dataset("art", split="validation")

Generating validation split:   0%|          | 0/1532 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/169654 [00:00<?, ? examples/s]

In [ ]:
test_test = datasets.load_dataset("art", split="validation[:3]")

In [ ]:
test_0_300_data = datasets.load_dataset('art', split='validation[:300]')
test_300_600_data = datasets.load_dataset('art', split='validation[300:600]')
test_600_900_data = datasets.load_dataset('art', split='validation[600:900]')
test_900_1200_data = datasets.load_dataset('art', split='validation[900:1200]')
test_1200_1530_data = datasets.load_dataset('art', split='validation[1200:]')

In [ ]:
chain_of_thoughts = ["Albert, being a weight loss guru, likely engaged in regular and possibly intense exercise. Dying on the last mile suggests he was pushing himself physically. This fits with hypothesis 1 that he increased his exercise regimen. Stopping exercise doesn’t explain the exertion implied.",
                     "Billy had a cat named Mittens, and cats are known to prefer boxes over beds. Mittens ignoring the bed suggests it was intended for her. Hypothesis 2, that Billy made a special bed for Mittens, fits better than hypothesis 1, which involves Cody, who is not mentioned elsewhere.",
                     "Andrew's dedication and hard work suggest he was very firm and strict in his planning. Becoming less rigid indicates a significant event caused this change. Hypothesis 1 explains that Andrew made his daughter cry by being too firm, leading him to reconsider his rigidity. Hypothesis 2 is not logical and lacks an explanation.",
                     "Amy needed to gather her courage, suggesting she was anxious about the party. If the only person she knew was Ella, it explains her anxiety and need for courage. Both hypotheses are similar, but hypothesis 2 directly states Ella was the only person Amy knew, aligning more clearly with Amy's feelings.",
                     "Jen staying up until 3am suggests she was pressed for time to complete her paper. This indicates she may have procrastinated. Hypothesis 1, that she put off finishing her paper to watch TV, explains why she had to stay up late. Hypothesis 2 implies she prioritized her paper, which doesn’t fit with the need to stay up late."]

In [53]:
def chain_of_thought_prompt_text(examples, chain_of_thoughts):
    """
    Generates a prompt text for chain of thought reasoning.

    Args:
        examples : A dataset containing the examples for chain of thought reasoning.
            It should have the following keys:
            - "observation_1": A list of strings representing the first observation.
            - "observation_2": A list of strings representing the second observation.
            - "hypothesis_1": A list of strings representing the first hypothesis.
            - "hypothesis_2": A list of strings representing the second hypothesis.
            - "label": A list of integers representing the label for each example.

        chain_of_thoughts (list): A list of strings representing the chain of thought for each example.

    Returns:
        str: The generated prompt text.

    """
    
    few_shot_prompt = []

    for i in range(len(examples["observation_1"])):
        few_shot_prompt.append("\nQuestion: \nGiven these two observations: " + examples["observation_1"][i]+ " "
                               + examples["observation_2"][i])
        few_shot_prompt.append("What is the most plausible explanation ?")
        few_shot_prompt.append("Hypothesis 1: "+ examples["hypothesis_1"][i])
        few_shot_prompt.append("Hypothesis 2: " +examples["hypothesis_2"][i])
        few_shot_prompt.append("Answer:")
        few_shot_prompt.append(chain_of_thoughts[i] + "\n\nThe answer is hypothesis " + str(examples["label"][i]))

    return "\n".join(few_shot_prompt).strip()

print(chain_of_thought_prompt_text(train_data[0:5], chain_of_thoughts))

Question: 
Given these two observations: Albert was a weight loss guru. He died of a heart attack on the last mile.
What is the most plausible explanation ?
Hypothesis 1: Albert increased his exercise regimen.
Hypothesis 2: Albert stopped his exercise regimen.
Answer:
Albert, being a weight loss guru, likely engaged in regular and possibly intense exercise. Dying on the last mile suggests he was pushing himself physically. This fits with hypothesis 1 that he increased his exercise regimen. Stopping exercise doesn’t explain the exertion implied.

The answer is hypothesis 1

Question: 
Given these two observations: Billy had a cat named Mittens. Mittens ignored the bed and slept in the box.
What is the most plausible explanation ?
Hypothesis 1: Billy made a special bed for Cody to sleep in.
Hypothesis 2: Billy made a special bed for Mittens to sleep in.
Answer:
Billy had a cat named Mittens, and cats are known to prefer boxes over beds. Mittens ignoring the bed suggests it was intended f

In [ ]:
def few_shot_prompt_text(examples):
    """
    Generate a few-shot prompt text based on the given examples.

    Args:
        examples: A dataset containing the examples with the following keys:
            - observation_1: List of strings representing the first observation.
            - observation_2: List of strings representing the second observation.
            - hypothesis_1: List of strings representing the first hypothesis.
            - hypothesis_2: List of strings representing the second hypothesis.
            - label: List of integers representing the label for each example.

    Returns:
        str: The generated few-shot prompt text.

    """
    
    few_shot_prompt = []

    for i in range(len(examples["observation_1"])):
        few_shot_prompt.append("\nQuestion: \nGiven these two observations: " + examples["observation_1"][i]+ " "
                               + examples["observation_2"][i])
        few_shot_prompt.append("What is the most plausible explanation ?")
        few_shot_prompt.append("Hypothesis 1: "+ examples["hypothesis_1"][i])
        few_shot_prompt.append("Hypothesis 2: " +examples["hypothesis_2"][i])
        few_shot_prompt.append("Answer:")
        few_shot_prompt.append("The most plausible explanation is hypothesis " + str(examples["label"][i]))

    return "\n".join(few_shot_prompt).strip()

print(few_shot_prompt_text(train_data[0:3]))

Question: 
Given these two observations: Albert was a weight loss guru. He died of a heart attack on the last mile.
What is the most plausible explanation ?
Hypothesis 1: Albert increased his exercise regimen.
Hypothesis 2: Albert stopped his exercise regimen.
Answer:
The most plausible explanation is hypothesis 1

Question: 
Given these two observations: Billy had a cat named Mittens. Mittens ignored the bed and slept in the box.
What is the most plausible explanation ?
Hypothesis 1: Billy made a special bed for Cody to sleep in.
Hypothesis 2: Billy made a special bed for Mittens to sleep in.
Answer:
The most plausible explanation is hypothesis 2

Question: 
Given these two observations: Andrew was very dedicated and hardworking. Andrew became less rigid about his planning after that.
What is the most plausible explanation ?
Hypothesis 1: Andrew accidentally made his daughter cry when he was firm.
Hypothesis 2: doesnt make sense.
Answer:
The most plausible explanation is hypothesis 1


In [ ]:
def entry_to_prompt_text(entry):
  """
  Converts an entry dictionary into a formatted prompt text.

  Args:
    entry (dict): A dictionary containing the following keys:
      - observation_1 (str): The first observation.
      - observation_2 (str): The second observation.
      - hypothesis_1 (str): The first hypothesis.
      - hypothesis_2 (str): The second hypothesis.

  Returns:
    str: The formatted prompt text.
  """

  prompt = []
  prompt.append("\nQuestion: \nGiven these two observations: " + entry["observation_1"]+ " "
                + entry["observation_2"])
  prompt.append("What is the most plausible explanation ?")
  prompt.append("Hypothesis 1: "+ entry["hypothesis_1"])
  prompt.append("Hypothesis 2: " +entry["hypothesis_2"])
  prompt.append("Answer:")

  return "\n".join(prompt).strip()

print(entry_to_prompt_text(train_data[0]))

Question: 
Given these two observations: Albert was a weight loss guru. He died of a heart attack on the last mile.
What is the most plausible explanation ?
Hypothesis 1: Albert increased his exercise regimen.
Hypothesis 2: Albert stopped his exercise regimen.
Answer:


In [ ]:
def save_answers(answers, path):
  """
  Saves the answers to a JSON file.

  Args:
    answers (list): A list containing the answers.
    path (str): The path to save the answers.

  Returns:
    None
  """

  results = {
      'model': "meta-llama/Llama-3-70b-chat-hf",
      'dataset': 'art_validation',
      'answers': answers,
  }

  with open(path, 'w') as f:
    json.dump(results, f, indent=4)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

def classification_analysis(predictions, true_labels, incorrect_indices, unknown_indices, report_path):
  """
  Performs classification analysis and saves the results to a JSON file.

  Args:
    predictions (list): A list of predicted labels.
    true_labels (list): A list of true labels.
    incorrect_indices (list): A list of indices where the predictions are incorrect.
    unknown_indices (list): A list of indices where the predictions are unknown.
    report_path (str): The path to save the classification report.

  Returns:
    None
  """

  filtered_true_labels = [label for label, pred in zip(true_labels, predictions) if pred is not None]
  filtered_predictions = [pred for pred in predictions if pred is not None]

  accuracy = accuracy_score(filtered_true_labels, filtered_predictions)
  f1 = f1_score(filtered_true_labels, filtered_predictions, average='weighted')
  report = classification_report(filtered_true_labels, filtered_predictions)

  print(f"Accuracy: {accuracy}")
  print(f"F1 Score: {f1}")
  print(f"Classification Report:\n{report}")

  results = {
      'model': "meta-llama/Llama-3-70b-chat-hf",
      'dataset': 'art_validation',
      'true_labels': true_labels,
      'predictions': predictions,
      'incorrect_indices': incorrect_indices,
      'unknown_indices': unknown_indices,
      'accuracy': accuracy,
      'f1_score': f1,
      'classification_report': report
  }

  with open(report_path, 'w') as f:
    json.dump(results, f, indent=4)

In [ ]:
def save_chunck(predictions, true_labels, incorrect_indices, unknown_indices, answers, chunk, report_path):
  """
  Saves the results of a chunk to a JSON file.

  Args:
    predictions (list): A list of predicted labels.
    true_labels (list): A list of true labels.
    incorrect_indices (list): A list of indices where the predictions are incorrect.
    unknown_indices (list): A list of indices where the predictions are unknown.
    answers (list): A list containing the text answers.
    chunk (str): The name of the chunk.
    report_path (str): The path to save the results.

  Returns:
    None
  """
  results = {
      'model': "meta-llama/Llama-3-70b-chat-hf",
      'dataset': chunk,
      'true_labels': true_labels,
      'predictions': predictions,
      'incorrect_indices': incorrect_indices,
      'unknown_indices': unknown_indices,
      'answers': answers,
  }

  with open(report_path, 'w') as f:
    json.dump(results, f, indent=4)

In [ ]:
def generate(prompt, client):
    """
    Generate a completion for the given prompt using the Together API.

    Args:
        prompt (str): The prompt text for the completion.
        client (Together): The Together client object.
        
    Returns:
        str: The generated completion text.
    """

    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(model="meta-llama/Llama-3-70b-chat-hf",
                                              messages=messages,
                                              max_tokens=256,
                                              temperature=0.6,
                                              top_p=0.9)

    return response.choices[0].message.content

In [ ]:
import re
def evaluate_zero_cot(test_data):
  """
  Evaluate the zero-shot chain of thought reasoning task.

  Args:
    test_data (Dataset): The test data.

  Returns:
    predictions (list): A list of predicted labels.
    true_labels (list): A list of true labels.
    incorrect_indices (list): A list of indices where the predictions are incorrect.
    unknown_indices (list): A list of indices where the predictions are unknown.
    answers (list): A list containing the text answers.
  """
  
  pattern = r"hypothesis (\d+)"

  true_labels = []
  predictions = []
  incorrect_indices = []
  unknown_indices =[]
  answers = []
  total, correct, unknown = 0, 0, 0

  for i, example in enumerate(test_data):

    total += 1

    prompt = entry_to_prompt_text(example) + " Let’s think step by step and determine which one is the most plausible hypothesis.\n"

    answer = generate(prompt, client)
    answer = answer.split("Question:")[0] if "Question:" in answer else answer
    answers.append(answer)
    matches = re.findall(pattern, answer, re.IGNORECASE)

    if matches:
      pred = int(matches[-1])
      predictions.append(pred)
      true_label = example['label']
      true_labels.append(true_label)

      if pred != true_label:
        incorrect_indices.append(i)
      else:
        correct+=1

    else:
      print(answer)
      unknown+=1
      predictions.append(None)
      true_labels.append(example['label'])
      unknown_indices.append(i)

    print(" *","Stats: ", f"{round(correct/total*100,2)}% correct, ({total} total, {correct} correct, {unknown} unknown)")

  return predictions, true_labels, incorrect_indices, unknown_indices, answers

In [50]:
def evaluate(test_data, few_shot_examples):
  """
  Evaluate the zero-shot and few-shot reasoning task depending on the content of few_shot_examples.

  Args:
    test_data (Dataset): The test data.
    few_shot_examples (str): The few-shot examples.

  Returns:
    predictions (list): A list of predicted labels.
    true_labels (list): A list of true labels.
    incorrect_indices (list): A list of indices where the predictions are incorrect.
    unknown_indices (list): A list of indices where the predictions are unknown.
    answers (list): A list containing the text answers.
  """

  pattern = r"hypothesis (\d+)"

  true_labels = []
  predictions = []
  incorrect_indices = []
  unknown_indices =[]
  answers = []
  total, correct, unknown = 0, 0, 0

  for i, example in enumerate(test_data):

    total += 1

    if few_shot_examples:
      prompt = few_shot_examples + "\n\n" + entry_to_prompt_text(example) + "\n"
    else:
      prompt = entry_to_prompt_text(example) + "\n"

    answer = generate(prompt, client)
    answer = answer.split("Question:")[0] if "Question:" in answer else answer
    answers.append(answer)
    matches = re.findall(pattern, answer, re.IGNORECASE)

    if matches:
      pred = int(matches[-1])
      predictions.append(pred)
      true_label = example['label']
      print(true_label)
      true_labels.append(true_label)

      if pred != true_label:
        incorrect_indices.append(i)
      else:
        correct+=1

    else:
      print(answer)
      unknown+=1
      predictions.append(None)
      true_labels.append(example['label'])
      unknown_indices.append(i)

    print(answer)
    print(" *","Stats: ", f"{round(correct/total*100,2)}% correct, ({total} total, {correct} correct, {unknown} unknown)")

  return predictions, true_labels, incorrect_indices, unknown_indices, answers

In [ ]:
def evaluate_self_consistency(test_data, few_shot_examples, k):
  """
  Evaluate the self-consistency task and pick the most frequent hypothesis in the k sampled paths.

  Args:
    test_data (Dataset): The test data.
    few_shot_examples (str): The few-shot examples.
    k (int): The number of samples to generate for each example.
  
  Returns:
    predictions (list): A list of predicted labels.
    true_labels (list): A list of true labels.
    incorrect_indices (list): A list of indices where the predictions are incorrect.
    unknown_indices (list): A list of indices where the predictions are unknown.
    answers (list): A list containing the text answers.
  """

  pattern = r"hypothesis (\d+)"

  true_labels = []
  predictions = []
  incorrect_indices = []
  unknown_indices =[]
  answers =[]
  total, correct, unknown = 0, 0, 0

  for i, example in enumerate(test_data):

    if few_shot_examples:
      prompt = few_shot_examples + "\n\n" + entry_to_prompt_text(example) + "\n"
    else:
      prompt = entry_to_prompt_text(example) + "\n"

    total += 1
    preds = []
    k_answers = []
    for j in range(k):
      answer = generate(prompt, client)
      answer = answer.split("Question:")[0] if "Question:" in answer else answer
      k_answers.append(answer)
      matches = re.findall(pattern, answer, re.IGNORECASE)
      if matches:
        pred = int(matches[-1])
      else:
        pred = None

      preds.append(pred)

    final_pred = max(set(preds), key=preds.count)
    true_label = example['label']
    predictions.append(final_pred)
    true_labels.append(true_label)
    answers.append(k_answers)
    if final_pred == None:
      unknown_indices.append(i)
    elif final_pred == true_label:
      correct+=1
    else:
      incorrect_indices.append(i)

    print(" *","Stats: ", f"{round(correct/total*100,2)}% correct, ({total} total, {correct} correct, {unknown} unknown)")

  return predictions, true_labels, incorrect_indices, unknown_indices, answers

### Zero shot prompting

In [ ]:
from google.colab import userdata
import os

os.environ["TOGETHER_API_KEY"] = userdata.get('TOGETHER_API_KEY')
client = Together(api_key=os.environ.get("TOGETHER_API_KEY"))

In [58]:
predictions, true_labels, incorrect_indices, unknown_indices, answers = evaluate(test_data, "")
classification_analysis(predictions, true_labels, incorrect_indices, unknown_indices,
                        '/content/gdrive/MyDrive/CS4NLP/llama3_70b/llama3_70B_zero_shot_prompt.json')

Streaming output truncated to the last 5000 lines.

Hypothesis 2 is less plausible because it assumes that Chang looked outside and saw the volcano was still, but the observations don't provide enough information to support this claim.
 * Stats:  48.36% correct, (943 total, 456 correct, 57 unknown)
2
The most plausible explanation is:

Hypothesis 2: Alice practiced the corner kick for hours every day.

This hypothesis is more plausible because it provides a logical connection between Alice's desire to learn how to kick a corner kick and her subsequent success in the game. It suggests that Alice put in the effort to improve her skills, which led to her being able to execute the corner kick effectively in the game.

Hypothesis 1, on the other hand, is unrelated to the context of the corner kick and seems out of place. It's unlikely that Alice asking her coach to kiss her would have any bearing on her ability to kick a corner kick.
 * Stats:  48.31% correct, (944 total, 456 correct, 57 un

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [59]:
save_answers(answers, '/content/gdrive/MyDrive/CS4NLP/llama3_70b/llama3_70B_zero_shot_prompt_answers.json')

### 5 shot prompting

In [43]:
examples = few_shot_prompt_text(train_data[0:5])
predictions, true_labels, incorrect_indices, unknown_indices, answers = evaluate(test_data, examples)
classification_analysis(predictions, true_labels, incorrect_indices, unknown_indices,
                        '/content/gdrive/MyDrive/CS4NLP/llama3_70b/llama3_70B_5shot_prompt.json')

1
 * Stats:  100.0% correct, (1 total, 1 correct, 0 unknown)
1
 * Stats:  100.0% correct, (2 total, 2 correct, 0 unknown)
1
 * Stats:  100.0% correct, (3 total, 3 correct, 0 unknown)
1
 * Stats:  75.0% correct, (4 total, 3 correct, 0 unknown)
1
 * Stats:  60.0% correct, (5 total, 3 correct, 0 unknown)
2
 * Stats:  66.67% correct, (6 total, 4 correct, 0 unknown)
1
 * Stats:  71.43% correct, (7 total, 5 correct, 0 unknown)
2
 * Stats:  75.0% correct, (8 total, 6 correct, 0 unknown)
2
 * Stats:  77.78% correct, (9 total, 7 correct, 0 unknown)
1
 * Stats:  80.0% correct, (10 total, 8 correct, 0 unknown)
1
 * Stats:  81.82% correct, (11 total, 9 correct, 0 unknown)
1
 * Stats:  83.33% correct, (12 total, 10 correct, 0 unknown)
2
 * Stats:  84.62% correct, (13 total, 11 correct, 0 unknown)
1
 * Stats:  85.71% correct, (14 total, 12 correct, 0 unknown)
1
 * Stats:  80.0% correct, (15 total, 12 correct, 0 unknown)
2
 * Stats:  81.25% correct, (16 total, 13 correct, 0 unknown)
1
 * Stats:  76.4

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [48]:
save_answers(answers, '/content/gdrive/MyDrive/CS4NLP/llama3_70b/llama3_70B_5shot_prompt_answers.json')

### Zero shot chain of thought prompting

In [62]:
predictions, true_labels, incorrect_indices, unknown_indices, answers = evaluate_zero_cot(test_data)
classification_analysis(predictions, true_labels, incorrect_indices, unknown_indices,
                        '/content/gdrive/MyDrive/CS4NLP/llama3_70b/llama3_70B_zero_shot_CoT_prompt.json')

 * Stats:  100.0% correct, (1 total, 1 correct, 0 unknown)
 * Stats:  50.0% correct, (2 total, 1 correct, 0 unknown)
 * Stats:  66.67% correct, (3 total, 2 correct, 0 unknown)
 * Stats:  75.0% correct, (4 total, 3 correct, 0 unknown)
 * Stats:  60.0% correct, (5 total, 3 correct, 0 unknown)
 * Stats:  66.67% correct, (6 total, 4 correct, 0 unknown)
 * Stats:  71.43% correct, (7 total, 5 correct, 0 unknown)
 * Stats:  75.0% correct, (8 total, 6 correct, 0 unknown)
 * Stats:  77.78% correct, (9 total, 7 correct, 0 unknown)
 * Stats:  80.0% correct, (10 total, 8 correct, 0 unknown)
 * Stats:  72.73% correct, (11 total, 8 correct, 0 unknown)
 * Stats:  75.0% correct, (12 total, 9 correct, 0 unknown)
 * Stats:  76.92% correct, (13 total, 10 correct, 0 unknown)
 * Stats:  78.57% correct, (14 total, 11 correct, 0 unknown)
 * Stats:  73.33% correct, (15 total, 11 correct, 0 unknown)
 * Stats:  75.0% correct, (16 total, 12 correct, 0 unknown)
 * Stats:  76.47% correct, (17 total, 13 correct, 0 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [63]:
save_answers(answers, '/content/gdrive/MyDrive/CS4NLP/llama3_70b/llama3_70B_zero_shot_CoT_prompt_answers.json')

### 5 shot chain of thought prompting

In [54]:
examples = chain_of_thought_prompt_text(train_data[0:5], chain_of_thoughts)
predictions, true_labels, incorrect_indices, unknown_indices, answers = evaluate(test_data, examples)
classification_analysis(predictions, true_labels, incorrect_indices, unknown_indices,
                        '/content/gdrive/MyDrive/CS4NLP/llama3_70b/llama3_70B_5_shot_CoT_prompt.json')

Streaming output truncated to the last 5000 lines.
 * Stats:  31.47% correct, (626 total, 197 correct, 23 unknown)
2
The answer is hypothesis 2.

Explanation:
Liz got bored of the game, which implies that she was playing a game with the 5-year-old boy. If she never played board games with him (hypothesis 1), it wouldn't explain why she got bored of the game. On the other hand, hypothesis 2, that she decided to play a board game with the boy, fits well with the observation that she got bored of the game, suggesting that the game was not engaging enough for her.
 * Stats:  31.58% correct, (627 total, 198 correct, 23 unknown)
2
The answer is hypothesis 2.

Explanation: Tanya wanted to mount her degrees on the wall, but her internet start-up manager stopped her. This suggests that the manager had authority over the workspace and was concerned about the appearance of the office. Hypothesis 2, that the boss didn't allow clutter on the walls, is a plausible explanation for why the manager sto

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [55]:
save_answers(answers, '/content/gdrive/MyDrive/CS4NLP/llama3_70b/llama3_70B_5_shot_CoT_prompt_answers.json')

### 5 shot chain of thought prompting with self-consistency

In [60]:
examples = chain_of_thought_prompt_text(train_data[0:5], chain_of_thoughts)
predictions, true_labels, incorrect_indices, unknown_indices, answers = evaluate_self_consistency(test_data, examples, 5)
classification_analysis(predictions, true_labels, incorrect_indices, unknown_indices,
                        '/content/gdrive/MyDrive/CS4NLP/llama3_70b/llama3_70B_5_shot_CoT_self_cons_prompt.json')

 * Stats:  100.0% correct, (1 total, 1 correct, 0 unknown)
 * Stats:  50.0% correct, (2 total, 1 correct, 0 unknown)
 * Stats:  66.67% correct, (3 total, 2 correct, 0 unknown)
 * Stats:  50.0% correct, (4 total, 2 correct, 0 unknown)
 * Stats:  60.0% correct, (5 total, 3 correct, 0 unknown)
 * Stats:  50.0% correct, (6 total, 3 correct, 0 unknown)
 * Stats:  42.86% correct, (7 total, 3 correct, 0 unknown)
 * Stats:  37.5% correct, (8 total, 3 correct, 0 unknown)
 * Stats:  33.33% correct, (9 total, 3 correct, 0 unknown)
 * Stats:  30.0% correct, (10 total, 3 correct, 0 unknown)
 * Stats:  36.36% correct, (11 total, 4 correct, 0 unknown)
 * Stats:  33.33% correct, (12 total, 4 correct, 0 unknown)
 * Stats:  30.77% correct, (13 total, 4 correct, 0 unknown)
 * Stats:  28.57% correct, (14 total, 4 correct, 0 unknown)
 * Stats:  26.67% correct, (15 total, 4 correct, 0 unknown)
 * Stats:  25.0% correct, (16 total, 4 correct, 0 unknown)
 * Stats:  23.53% correct, (17 total, 4 correct, 0 unkno

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [61]:
save_answers(answers, '/content/gdrive/MyDrive/CS4NLP/llama3_70b/llama3_70B_5_shot_CoT_self_cons_prompt_answers.json')